# Kaggle Zillow Preprocessing + XGBoost

###### 0.0660352

In [1]:
import numpy as np
import pandas as pd
import time
from datetime import datetime

from sami_function import missing_ratio

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
import gc
from sklearn import linear_model

%matplotlib inline

In [2]:
seed = 42

In [3]:
def create_newFeatures(dataframe):
    """
    Create new features for Zillow dataframe

    """

    if 'transactiondate' in dataframe.columns:
        dataframe['transactiondate'] =  pd.to_datetime(dataframe['transactiondate'])
        dataframe['transaction_year'] = dataframe.transactiondate.dt.year.astype(np.int16)
        dataframe['transaction_month'] = dataframe.transactiondate.dt.month.astype(np.int8)
        # dataframe['transaction_day'] = dataframe.transactiondate.dt.weekday.astype(np.int8)
        # dataframe['transaction_quarter'] = dataframe.transactiondate.dt.quarter.astype(np.int8)
        del dataframe['transactiondate']
    else:
        df_date = pd.DataFrame({'transaction_year': [2016, 2016, 2016, 2017, 2017, 2017],
                             'transaction_month': [10, 11, 12, 10, 11, 12]})
        df_date['tmp'] = 1
        dataframe['tmp'] = 1
        dataframe = pd.merge(dataframe, df_date, on='tmp')
        del dataframe['tmp']
        # dataframe['transaction_quarter'] = str(str(dataframe['transaction_year'])+'-'+str(dataframe['transaction_month'])+'-01')
        # dataframe['transaction_quarter'] = pd.to_datetime(dataframe['transaction_quarter']).dt.quarter.astype(np.int8)
                             
    dataframe['rawcensustractandblock_states'] = dataframe.rawcensustractandblock.astype(str).apply(lambda x: x[:1]).astype(np.int8)
    dataframe['rawcensustractandblock_countries'] = dataframe.rawcensustractandblock.astype(str).apply(lambda x: x[1:4]).astype(np.int8)
    dataframe['rawcensustractandblock_tracts'] = dataframe.rawcensustractandblock.astype(str).apply(lambda x: x[4:11]).astype(np.float64)
    dataframe['rawcensustractandblock_blocks'] = dataframe.rawcensustractandblock.astype(str).apply(lambda x: 0 if x[11:] == '' else x[11:]).astype(np.int8)
    
    #--- how old is the house? ---
    dataframe['yearbuilt'] = dataframe['yearbuilt'].fillna(2016).astype(np.int16)
    dataframe['house_age'] = dataframe['transaction_year'].astype(np.int16) - dataframe['yearbuilt'].astype(np.int16)

    #--- how many rooms are there? ---
    dataframe['bathroomcnt'] = dataframe['bathroomcnt'].fillna(1)
    dataframe['bedroomcnt'] = dataframe['bedroomcnt'].fillna(1)
    dataframe['tot_rooms'] = dataframe['bathroomcnt'] + dataframe['bedroomcnt']

    #--- does the house have A/C? ---
    dataframe['airconditioningtypeid'] = dataframe['airconditioningtypeid'].fillna(5)
    dataframe['AC'] = np.where(dataframe['airconditioningtypeid']>0, 1, 0)

    #--- Does the house have a deck? ---
    dataframe['decktypeid'] = dataframe['decktypeid'].fillna(0)
    dataframe['deck'] = np.where(dataframe['decktypeid']>0, 1, 0)
    dataframe.drop('decktypeid', axis=1, inplace=True)

    #--- does the house have a heating system? ---
    dataframe['heatingorsystemtypeid'] = dataframe['heatingorsystemtypeid'].fillna(13)
    dataframe['heating_system'] = np.where(dataframe['heatingorsystemtypeid']>0, 1, 0)

    #--- does the house have a garage? ---
    dataframe['garagecarcnt'] = dataframe['garagecarcnt'].fillna(0)
    dataframe['garage'] = np.where(dataframe['garagecarcnt']>0, 1, 0)

    #--- does the house come with a patio? ---
    dataframe['yardbuildingsqft17'] = dataframe['yardbuildingsqft17'].fillna(0)
    dataframe['patio'] = np.where(dataframe['yardbuildingsqft17']>0, 1, 0)

    #--- does the house have a pool?
    dataframe['pooltypeid10'] = dataframe.pooltypeid10.fillna(0).astype(np.int8)
    dataframe['pooltypeid7'] = dataframe.pooltypeid7.fillna(0).astype(np.int8)
    dataframe['pooltypei2'] = dataframe.pooltypeid2.fillna(0).astype(np.int8)
    dataframe['pool'] = dataframe['pooltypeid10'] | dataframe['pooltypeid7'] | dataframe['pooltypeid2']

    #--- does the house have all of these? -> spa/hot-tub/pool, A/C, heating system , garage, patio
    dataframe['exquisite'] = dataframe['pool'] + dataframe['patio'] + dataframe['garage'] + dataframe['heating_system'] + dataframe['AC']

    #--- Features based on location ---
    dataframe['x_loc'] = np.cos(dataframe['latitude']) * np.cos(dataframe['longitude'])
    dataframe['y_loc'] = np.cos(dataframe['latitude']) * np.sin(dataframe['longitude'])
    dataframe['z_loc'] = np.sin(dataframe['latitude'])

    return dataframe

In [4]:
def memory_reduce(dataframe):

    #--- Memory usage of entire dataframe ---
    mem = dataframe.memory_usage(index=True).sum()
    print("Initial size {:.2f} MB".format(mem/ 1024**2))

    #--- List of columns that cannot be reduced in terms of memory size ---
    count = 0
    for c in dataframe.columns:
        if dataframe[c].dtype == object:
            count+=1
    print('There are {} columns that cannot be reduced'.format(count))

    count = 0
    for c in dataframe.columns:

        if dataframe[c].dtype in ['int8', 'int16', 'int32', 'int64']:
            
            if (np.iinfo(np.int8).min < dataframe[c].min()) and (dataframe[c].max() < np.iinfo(np.int8).max):
                count+=1
                dataframe[c] = dataframe[c].fillna(0).astype(np.int8)
            
            if (np.iinfo(np.int16).min < dataframe[c].min()) and (dataframe[c].max() < np.iinfo(np.int16).max) and ((np.iinfo(np.int8).min > dataframe[c].min()) or (dataframe[c].max() > np.iinfo(np.int8).max)):
                count+=1
                dataframe[c] = dataframe[c].fillna(0).astype(np.int16)
            
            if (np.iinfo(np.int32).min < dataframe[c].min()) and (dataframe[c].max() < np.iinfo(np.int32).max) and ((np.iinfo(np.int16).min > dataframe[c].min()) or (dataframe[c].max() > np.iinfo(np.int16).max)):
                count+=1
                dataframe[c] = dataframe[c].fillna(0).astype(np.int32)
            
            if (np.iinfo(np.int64).min < dataframe[c].min()) and (dataframe[c].max() < np.iinfo(np.int64).max) and ((np.iinfo(np.int32).min > dataframe[c].min()) or (dataframe[c].max() > np.iinfo(np.int32).max)):
                count+=1
                dataframe[c] = dataframe[c].fillna(0).astype(np.int64)

        if dataframe[c].dtype in ['float16', 'float32', 'float64']:
            
            if (np.finfo(np.float16).min < dataframe[c].min()) and (dataframe[c].max() < np.finfo(np.float16).max):
                count+=1
                dataframe[c] = dataframe[c].fillna(0).astype(np.float16)
            
            if (np.finfo(np.float32).min < dataframe[c].min()) and (dataframe[c].max() < np.finfo(np.float32).max) and ((np.finfo(np.float16).min > dataframe[c].min()) or (dataframe[c].max() > np.finfo(np.float16).max)):
                count+=1
                dataframe[c] = dataframe[c].fillna(0).astype(np.float32)
            
            if (np.finfo(np.float64).min < dataframe[c].min()) and (dataframe[c].max() < np.finfo(np.float64).max) and ((np.finfo(np.float32).min > dataframe[c].min()) or (dataframe[c].max() > np.finfo(np.float32).max)):
                count+=1
                dataframe[c] = dataframe[c].fillna(0).astype(np.float64)

    print('There are {} columns reduced'.format(count))

    #--- Let us check the memory consumed again ---
    mem = dataframe.memory_usage(index=True).sum()
    print("Final size {:.2f} MB".format(mem/ 1024**2))

    return dataframe

In [5]:
def data_preprocessing(dataframe):
    
    dataframe['rawcensustractandblock'] = dataframe.rawcensustractandblock.fillna(dataframe.rawcensustractandblock.mode()[0])
    
    dataframe['buildingclasstypeid'] = dataframe['buildingclasstypeid'].fillna(dataframe['buildingclasstypeid'].mode()[0])
    dataframe['storytypeid'] = dataframe['storytypeid'].fillna(dataframe['storytypeid'].mode()[0])
    dataframe['architecturalstyletypeid'] = dataframe['architecturalstyletypeid'].fillna(dataframe['architecturalstyletypeid'].mode()[0])
    dataframe['typeconstructiontypeid'] = dataframe['typeconstructiontypeid'].fillna(dataframe['typeconstructiontypeid'].mode()[0])

    dataframe['taxdelinquencyyear'] = dataframe['taxdelinquencyyear'].fillna(15).astype(np.int8)
    dataframe['taxdelinquencyyear'] = np.where(dataframe.taxdelinquencyyear < 18, 2000 + dataframe.taxdelinquencyyear.astype(np.int16), 1900 + dataframe.taxdelinquencyyear.astype(np.int16)).astype(np.int16)

    dataframe['taxamount'] = dataframe['taxamount'].fillna(dataframe['taxamount'].mean())

    for c in dataframe.columns:
        if 'squarefeet' in c or 'sqft' in c or 'size' in c or 'pooltypeid' in c or 'cnt' in c or 'nbr' in c or 'number' in c:
            dataframe[c] = dataframe[c].fillna(0)
    
    #--- drop out ouliers ---
#     dataframe = dataframe[dataframe['logerror'] > -0.4 ]
#     dataframe = dataframe[dataframe['logerror'] < 0.4 ]
    
#     # replace or drop values ???
#     for c in dataframe.columns:
#         if c == 'logerror':
#             ulimit = np.percentile(dataframe[c].values, 99)
#             llimit = np.percentile(dataframe[c].values, 1)
#             dataframe.loc[dataframe[c] > ulimit, [c]] = ulimit
#             dataframe.loc[dataframe[c] < llimit, [c]] = llimit
    
    for c in dataframe.dtypes[dataframe.dtypes == object].index.values:
        if len(dataframe[c].unique()) <= 2:
            dataframe[c] = dataframe[c].map({True: 1, 'Y': 1})
            dataframe[c] = dataframe[c].fillna(0)
            dataframe[c] = dataframe[c].astype(np.int8)
        else:
            dataframe[c] = dataframe[c].fillna(1)
            lbl = LabelEncoder()
            lbl.fit(list(dataframe[c].values))
            dataframe[c] = lbl.transform(list(dataframe[c].values)).astype(np.int8)

    return dataframe

### XGBoost

In [6]:
%%time
print('\nLoading data ...')

train =  pd.read_csv('../data/train_2016_v2.csv')
prop = pd.read_csv('../data/properties_2016.csv')
sample = pd.read_csv('../data/sample_submission.csv')


print('\nCreating new features ...')

df_train = pd.merge(train, prop, on='parcelid', how='left')
print('Shape train: {}'.format(df_train.shape))

del train; gc.collect()

df_train = create_newFeatures(df_train)


print('\nData preprocessing ...')

df_train = data_preprocessing(df_train)


print('\nReducing consumption memory ...')

df_train = memory_reduce(df_train)

# print('\nDropping columns ...')

# col_2_drop = list(missing_ratio(df_train, plot=False).index)
# df_train = df_train.drop(col_2_drop, axis=1)

print('\nCreating training set ...')

x_train = df_train.drop(['parcelid', 'logerror'], axis=1)  # , 'propertyzoningdesc', 'propertycountylandusecode'
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

y_mean = np.mean(y_train)
train_columns = x_train.columns
    
del df_train; gc.collect()

# split = 80000
# x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]
# # x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.12, random_state=seed)

print('\nBuilding DMatrix...')

# d_train = xgb.DMatrix(x_train, label=y_train)
# d_valid = xgb.DMatrix(x_valid, label=y_valid)

# del x_train, x_valid; gc.collect()

d_train = xgb.DMatrix(x_train, y_train)

del x_train; gc.collect()

print('\nTraining ...')

params = {
    'eta': 0.03,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'base_score': y_mean,
    'silent': 1,
    'seed': seed
}

#--- cross-validation ---
cv_result = xgb.cv(
                    params, 
                    d_train, 
                    nfold=10,
                    num_boost_round=1000,
                    early_stopping_rounds=100,
                    verbose_eval=10, 
                    show_stdv=False
                  )

num_boost_rounds = cv_result['test-mae-mean'].argmin()
mean_mae = cv_result['test-mae-mean'].min()

print("\n\tMAE {} for {} rounds".format(mean_mae, num_boost_rounds))


#--- train model ---
clf = xgb.train(dict(params), d_train, num_boost_round=num_boost_rounds)

# watchlist = [(d_train, 'train'), (d_valid, 'valid')]
# clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=100, verbose_eval=10)


# del d_train, d_valid
del d_train

print('\nBuilding test set ...')

sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')

del prop, sample; gc.collect()

p_test = []
batch_size = 100000
for batch in range(batch_size, df_test.shape[0]+batch_size, batch_size):
    
    print('\nWorking batch {}'.format(batch))
    
    df_test_batch = df_test[batch-batch_size:batch].copy()
    
    print('\nCreating new features ...')
    
    df_test_batch['rawcensustractandblock'] = df_test_batch.rawcensustractandblock.fillna(df_test.rawcensustractandblock.mode()[0])
    df_test_batch = df_test_batch.fillna(0)
    
    df_test_batch = create_newFeatures(df_test_batch)
    
    print('\nData preprocessing ...')

    df_test_batch = data_preprocessing(df_test_batch)


    print('\nReducing consumption memory ...')
    
    df_test_batch = memory_reduce(df_test_batch)

    x_test = df_test_batch[train_columns]

    del df_test_batch; gc.collect()

    d_test = xgb.DMatrix(x_test)

    del x_test; gc.collect()

    print('\nPredicting on test ...')

    p_test_batch = clf.predict(d_test)

    del d_test; gc.collect()
    
    [p_test.append(p) for p in p_test_batch]

i = 0
sub = pd.read_csv('../data/sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test[i::6]
    i = i + 1

print('\nWriting csv ...')
sub.to_csv('../submissions/xgb_{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False, float_format='%.4f')

print('\nPrediction available !!!')


Loading data ...


<string>:2: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.



Creating new features ...
Shape train: (90275, 60)

Data preprocessing ...

Reducing consumption memory ...
Initial size 41.84 MB
There are 0 columns that cannot be reduced
There are 76 columns reduced
Final size 14.64 MB

Creating training set ...
(90275, 76) (90275,)

Building DMatrix...

Training ...
[0]	train-mae:0.0684124	test-mae:0.0684197
[10]	train-mae:0.0681056	test-mae:0.0682036
[20]	train-mae:0.0679405	test-mae:0.0681218
[30]	train-mae:0.0678438	test-mae:0.0681107
[40]	train-mae:0.067768	test-mae:0.0681024
[50]	train-mae:0.0677012	test-mae:0.0681133
[60]	train-mae:0.0676389	test-mae:0.0681251
[70]	train-mae:0.0675867	test-mae:0.0681393
[80]	train-mae:0.0675326	test-mae:0.0681553
[90]	train-mae:0.0674807	test-mae:0.0681713
[100]	train-mae:0.0674322	test-mae:0.0681881
[110]	train-mae:0.0673908	test-mae:0.0682067
[120]	train-mae:0.0673463	test-mae:0.068227
[130]	train-mae:0.0673073	test-mae:0.0682564

	MAE 0.06810240000000001 for 40 rounds

Building test set ...

Working batch

ValueError: Unknown string format

In [ ]:
sub